In [ ]:
import os
import pandas as pd
import numpy as np
import csv
import re

In [2]:
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
 
# spacy for lemmatization
import spacy
 
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
#读入数据
group_1_del = pd.read_csv('D:\my research\group\group_1\group_1_final.csv',encoding='utf-8')
group_1_del.columns = ['0','content','date','time','user_id','cut_content','del']
group_1_del = group_1_del.drop(columns='0')
group_1_del.head()

,content,date,time,user_id,cut_content,del
0,#上海疫情##上海民生##就这一刻##返乡# 上海返乡潮开启，安徽、江苏和河南人最多，隔离和...,2022-04-24,2022-04-24 08:33:43,1000074972,# 上海 疫情 ## 上海 民生 ## 就 这 一刻 ## 返乡 # 上海 返乡 潮 开...,疫情 民生 一刻 返乡 返乡 潮 开启 安徽 江苏 河南人 隔离 做 核酸 钱 返乡 潮...
1,#上海疫情##上海民生##上海现状# .上海的现状为什么会变成这样我忍不住地继续瞎想外行领导...,2022-04-23,2022-04-23 05:46:14,1000074972,# 上海 疫情 ## 上海 民生 ## 上海 现状 # . 上海 的 现状 为什么 会 ...,疫情 民生 现状 现状 忍不住 瞎 想 外行 领导 内行 内行 闭嘴 同理 衍生 行业 ...
2,#上海疫情##上海民生# 无间道，罗生门，可能本来就是人生如戏吧，是我太认真了。算...,2022-04-22,2022-04-22 11:47:31,1000074972,# 上海 疫情 ## 上海 民生 # 无间道 ， 罗生门 ， 可能 ...,疫情 民生 无间道 罗生门 本来 人生 如戏 是我太 算了 干脆 眼...
3,#上海民生##上海疫情##上海团长# 还是你牛逼呀，团长。开个车照送，还是封控小区。,2022-04-22,2022-04-22 09:41:33,1000074972,# 上海 民生 ## 上海 疫情 ## 上海 团长 # 还是 你 牛 逼 呀 ， 团长 ...,民生 疫情 团长 牛 逼 团长 开个 车照 送 封控 小区
4,明康汇的采购员你成功地把你们老板一个月来的光辉形象給黑了。@明康汇 @市场监管 @江丄孤舟 ...,2022-04-21,2022-04-21 08:54:29,1000074972,明康汇 的 采购员 你 成功 地 把 你们 老板 一个月 来 的 光辉 形象 給黑 了 。 ...,明康汇 采购员 成功 老板 一个月 光辉 形象 給黑 明康汇 市场监管 江 丄 孤 ...


In [4]:
group_1_del.shape

(284773, 6)

In [5]:
import pyLDAvis.sklearn
import pyLDAvis
import pyLDAvis.gensim

In [6]:
#完善停用词表
new = (['顾魏','特别','越来越','好像','我要','刚刚','天天','林之校','啊啊啊','王一博','翟潇闻','真的','哈哈哈','感觉',
       '不到','一点','呜呜','哈哈哈哈','好好','这是','时代','少年','余生','指教','本来','晚上','天天','不想'])

In [7]:
group_1_del.isnull().any(axis=0)

content        False
date           False
time           False
user_id        False
cut_content    False
del             True
dtype: bool

In [8]:
# 按行删除：存在空值，即删除该行
un = group_1_del.dropna(axis=0, how='any')
un.shape

(284267, 6)

In [9]:
un.isnull().any(axis=0)

content        False
date           False
time           False
user_id        False
cut_content    False
del            False
dtype: bool

In [12]:
aaa = []
for line in un['del']:
    line_u = ' '.join(str(line).split())
    line_uu = line_u.split()#将一个元素按空格拆分成多个元素
    outstr = []
    for words in line_uu:
        if words not in new:
            #if words in ['n','v','a','r','i']:
                outstr.append(words)
    aaa.append(outstr)

In [13]:
aaa

[['疫情',
  '民生',
  '一刻',
  '返乡',
  '返乡',
  '潮',
  '开启',
  '安徽',
  '江苏',
  '河南人',
  '隔离',
  '做',
  '核酸',
  '钱',
  '返乡',
  '潮',
  '开启',
  '滞留',
  '人员',
  '陆续',
  '返乡',
  '安徽',
  '江苏',
  '河南',
  '返乡',
  '人员'],
 ['疫情',
  '民生',
  '现状',
  '现状',
  '忍不住',
  '瞎',
  '想',
  '外行',
  '领导',
  '内行',
  '内行',
  '闭嘴',
  '同理',
  '衍生',
  '行业',
  '一条',
  '汉奸',
  '太',
  '抗战',
  '那会',
  '太',
  '敏感',
  '我乎',
  '审核',
  '说',
  '一条',
  '有人',
  '发国难财',
  '能量',
  '不小'],
 ['疫情',
  '民生',
  '无间道',
  '罗生门',
  '人生',
  '如戏',
  '是我太',
  '算了',
  '干脆',
  '眼一闭',
  '当作',
  '事情',
  '发生',
  '岁月',
  '静好',
  '叫唤'],
 ['民生', '疫情', '团长', '牛', '逼', '团长', '开个', '车照', '送', '封控', '小区'],
 ['明康汇',
  '采购员',
  '成功',
  '老板',
  '一个月',
  '光辉',
  '形象',
  '給黑',
  '明康汇',
  '市场监管',
  '江',
  '丄',
  '孤',
  '舟',
  '中国新闻社'],
 ['疫情', '民生', '物资', '黑市'],
 ['民生',
  '疫情',
  '外卖',
  '骑手',
  '好自为之',
  '骑手',
  '代购',
  '袋',
  '泡面',
  '鸡蛋',
  '收取',
  '元',
  '警方',
  '调查',
  '惊人',
  '发现',
  '最新消息',
  '证实',
  '赵某',
  '因涉嫌',
  '非法经营',
  '罪',
  '长宁',
  '警方',
  '依法',

In [18]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

In [15]:
dictionary = corpora.Dictionary(aaa)
corpus = [dictionary.doc2bow(tmp) for tmp in aaa]

In [20]:
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [25]:
# 困惑度的XY坐标
x_perplexity = [] 
y_perplexity = [] 
# 一致性的XY坐标
x_coherence = [] 
y_coherence = [] 
#id2word：就是上边构造的dictionary字典
#passes：模型的训练次数
for i in range(1, 31):     
    # 循环生成主题数为i的模型， passes代表模型训练次数，数据量太大了，搞小一点。     
    lda = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=i, id2word=dictionary, passes=10)     
    # 计算当前模型困惑度
    cur_perplexity = lda.log_perplexity(corpus_tfidf)     
    # 构造横纵坐标轴数据
    x_perplexity.append(i)
    y_perplexity.append(cur_perplexity) 
    # 计算一致性
    cv_tmp = CoherenceModel(model=lda, texts=aaa, dictionary=dictionary, coherence='c_v') 
    x_coherence.append(i) 
    y_coherence.append(cv_tmp.get_coherence()) 

KeyboardInterrupt: 

In [24]:
# 构造 TF-IDF
#tf_idf_vectorizer = TfidfVectorizer()
tf_idf_vectorizer = TfidfVectorizer(min_df = 0.001,max_df = 0.8)
tf_idf = tf_idf_vectorizer.fit_transform(aaa)

In [25]:
# 特征词列表
feature_names = tf_idf_vectorizer.get_feature_names()
# 特征词 TF-IDF 矩阵
matrix = tf_idf.toarray()
feature_names_df = pd.DataFrame(matrix,columns=feature_names)
print(feature_names_df)
feature_names_df

        一下子  一个个  一个多月       一个月   一人   一件   一份   一会  一会儿   一位  ...   高价   高考  \
0       0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1       0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2       0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3       0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4       0.0  0.0   0.0  0.544491  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...     ...  ...   ...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
284262  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
284263  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
284264  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
284265  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
284266  0.0  0.0   0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

         魔幻   鸡蛋   麻烦   黄瓜 

,一下子,一个个,一个多月,一个月,一人,一件,一份,一会,一会儿,一位,...,高价,高考,魔幻,鸡蛋,麻烦,黄瓜,默默,鼓励,鼻子,龚俊
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.544491,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284262,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
284263,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
284264,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
284265,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# 指定 lda 主题数
n_topics = 6
# 要输出的每个主题的前 n_top_words 个主题词数
n_top_words = 20

lda = LatentDirichletAllocation(
    n_components=n_topics, max_iter=50,
    learning_method='online',
    learning_offset=50.,
    random_state=0)
# 核心，给 LDA 喂生成的 TF-IDF 矩阵
lda.fit(tf_idf)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, n_components=6, random_state=0)

In [27]:
def top_words_data_frame(model: LatentDirichletAllocation,
                         tf_idf_vectorizer: TfidfVectorizer,
                         n_top_words: int) -> pd.DataFrame:
    '''
    求出每个主题的前 n_top_words 个词

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    tf_idf_vectorizer : sklearn 的 TfidfVectorizer
    n_top_words :前 n_top_words 个主题词

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    rows = []
    feature_names = tf_idf_vectorizer.get_feature_names()
    for topic in model.components_:
        top_words = [feature_names[i]
                     for i in topic.argsort()[:-n_top_words - 1:-1]]
        rows.append(top_words)
    columns = [f'topic {i+1}' for i in range(n_top_words)]
    df = pd.DataFrame(rows, columns=columns)

    return df


def predict_to_data_frame(model: LatentDirichletAllocation, X: np.ndarray) -> pd.DataFrame:
    '''
    求出文档主题概率分布情况

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    X : 词向量矩阵

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    # 求出给定文档的主题概率分布矩阵
    matrix = model.transform(X)
    columns = [f'P(topic {i+1})' for i in range(len(model.components_))]
    df = pd.DataFrame(matrix, columns=columns)
    return df

In [28]:
# 计算 n_top_words 个主题词
top_words_df = top_words_data_frame(lda, tf_idf_vectorizer, n_top_words)
top_words_df
# 保存 n_top_words 个主题词到 csv 文件中
#top_words_df.to_csv(top_words_csv_path, encoding='utf-8-sig', index=None)

# 转 tf_idf 为数组，以便后面使用它来对文本主题概率分布进行计算
X = tf_idf.toarray()

# 计算完毕主题概率分布情况
predict_df = predict_to_data_frame(lda, X)
predict_df
# 保存文本主题概率分布到 csv 文件中
#predict_df.to_csv(predict_topic_csv_path, encoding='utf-8-sig', index=None)

,P(topic 1),P(topic 2),P(topic 3),P(topic 4),P(topic 5),P(topic 6)
0,0.238608,0.103781,0.526083,0.043869,0.043855,0.043805
1,0.241819,0.579951,0.044544,0.044547,0.044601,0.044539
2,0.229953,0.047663,0.046950,0.047031,0.047127,0.581277
3,0.439222,0.057445,0.057091,0.057837,0.057239,0.331166
4,0.061208,0.694483,0.061077,0.061077,0.061077,0.061077
...,...,...,...,...,...,...
284262,0.618165,0.151378,0.104857,0.033257,0.033349,0.058994
284263,0.549691,0.229219,0.107761,0.027731,0.057769,0.027828
284264,0.595831,0.053068,0.029391,0.051089,0.241185,0.029436
284265,0.619227,0.032737,0.032565,0.032807,0.212417,0.070247


In [34]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel

In [29]:
pyLDAvis.enable_notebook()
data = pyLDAvis.sklearn.prepare(lda, tf_idf, tf_idf_vectorizer)
#让可视化可以在notebook内显示
pyLDAvis.show(data)

D:\Program_Files\anaconda\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [20/Oct/2022 21:06:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2022 21:06:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2022 21:08:02] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2022 21:08:02] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2022 21:08:02] "GET /LDAvis.js HTTP/1.1" 200 -



stopping Server...
